# Evaluating Segment Anything (SAM)

Here is an example for evaluating the automatic and interactive segmentation capability of SAM and `µsam` on differential interference contrast microscopic images from the Cell Tracking Challenge (DIC-C2DH-HeLa: HeLa cells on a flat glass - https://doi.org/10.1038/s41592-023-01879-y).

With the scripts below, you can validate the quality of the Segment Anything models to perform interactive segmentation.

We compare three settings in batched mode for all images:
1. Automatic Mask Generation (AMG): The "Segment Anything" feature where a grid of positive point prompts are sampled over the entire image to perform instance segmentation.
2. Automatic Instance Segmentation (AIS): The feature in `µsam` (Segment Anything for Microscopy) where we introduce an additional decoder to perform automatic instance segmentation.
3. Interactive Segmentation using Input Prompts derived from ground-truth objects.

## Let's check, where are you now?

**NOTE**: The scripts have been tested on our recommended open-source cloud servers, [Kaggle Notebooks](https://www.kaggle.com/code/).

In [1]:
import os
current_spot = os.getcwd()

if current_spot.startswith("/kaggle/working"):
    using_kaggle = True
    print("Kaggle says hi!")
    root_dir = "/kaggle/working"

else:
    if current_spot.startswith("/content"):
        using_colab = True
        print("Google Colab says hi!")
        print(" NOTE: The scripts have not been tested on Google Colab, you might need to adapt the installations a bit.")
        root_dir = "/content"

        # You might need to install condacolab on Google Colab to be able to install packages using conda / mamba
        # !pip install -q condacolab
        # import condacolab
        # condacolab.install()
    
    else:
        msg = "You are using a behind-the-scenes resource. Follow our installation instructions here:"
        msg += " https://computational-cell-analytics.github.io/micro-sam/micro_sam.html#installation"
        print(msg)
        root_dir = ""  # overwrite to set the root directory, where the data, checkpoints, and all relevant stuff will be stored

Kaggle says hi!


## Installation

Let's install all the relevant dependencies required for inference.

In [2]:
import tifffile
tifffile.__version__

'2023.12.9'

In [3]:
!git clone --quiet https://github.com/computational-cell-analytics/micro-sam.git
tmp_dir = os.path.join(root_dir, "micro-sam")
!pip install --quiet $tmp_dir

Branch 'u-eval' set up to track remote branch 'u-eval' from 'origin'.
Switched to a new branch 'u-eval'


In [4]:
!git clone --quiet https://github.com/constantinpape/torch-em.git
tmp_dir = os.path.join(root_dir, "torch-em")
!pip install --quiet $tmp_dir

In [5]:
!git clone --quiet https://github.com/constantinpape/elf.git
tmp_dir = os.path.join(root_dir, "elf")
!pip install --quiet $tmp_dir

In [6]:
!mamba install -q -y -c conda-forge nifty affogato zarr z5py
!pip uninstall -y --quiet qtpy  # qtpy is not supported in Kaggle / Google Colab, let's remove it to avoid errors.

warning  libmamba Cache file "/opt/conda/pkgs/cache/497deca9.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/09cdf8bf.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/c6f2354e.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/86b0f08d.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/c9ddbd6b.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/b121c3e7.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/47929eba.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/3e39a7aa.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/2ce54b42.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/4ea078d6.json" was modified by another program
Preparing 

## Importing the libraries

In [7]:
from glob import glob

import imageio.v3 as imageio
from matplotlib import pyplot as plt

from torch_em.util.util import get_random_colors

from micro_sam.util import get_sam_model
from micro_sam.evaluation import inference
from micro_sam.evaluation.model_comparison import _enhance_image
from micro_sam.sample_data import fetch_tracking_example_data, fetch_tracking_segmentation_data
from micro_sam.evaluation.evaluation import run_evaluation, run_evaluation_for_iterative_prompting

2024-05-04 21:46:57.737845: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 21:46:57.737940: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 21:46:57.859988: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Let's download the dataset

In [8]:
DATA_FOLDER = os.path.join(root_dir, "data")
os.makedirs(DATA_FOLDER, exist_ok=True)

# This will download the image and segmentation data for training.
image_dir = fetch_tracking_example_data(DATA_FOLDER)
segmentation_dir = fetch_tracking_segmentation_data(DATA_FOLDER)

100%|█████████████████████████████████████| 41.5M/41.5M [00:00<00:00, 49.0GB/s]
Extracting 'DIC-C2DH-HeLa/01/t000.tif' from '/kaggle/working/data/DIC-C2DH-HeLa.zip' to '/kaggle/working/data/DIC-C2DH-HeLa.zip.unzip'
Extracting 'DIC-C2DH-HeLa/01/t001.tif' from '/kaggle/working/data/DIC-C2DH-HeLa.zip' to '/kaggle/working/data/DIC-C2DH-HeLa.zip.unzip'
Extracting 'DIC-C2DH-HeLa/01/t002.tif' from '/kaggle/working/data/DIC-C2DH-HeLa.zip' to '/kaggle/working/data/DIC-C2DH-HeLa.zip.unzip'
Extracting 'DIC-C2DH-HeLa/01/t003.tif' from '/kaggle/working/data/DIC-C2DH-HeLa.zip' to '/kaggle/working/data/DIC-C2DH-HeLa.zip.unzip'
Extracting 'DIC-C2DH-HeLa/01/t004.tif' from '/kaggle/working/data/DIC-C2DH-HeLa.zip' to '/kaggle/working/data/DIC-C2DH-HeLa.zip.unzip'
Extracting 'DIC-C2DH-HeLa/01/t005.tif' from '/kaggle/working/data/DIC-C2DH-HeLa.zip' to '/kaggle/working/data/DIC-C2DH-HeLa.zip.unzip'
Extracting 'DIC-C2DH-HeLa/01/t006.tif' from '/kaggle/working/data/DIC-C2DH-HeLa.zip' to '/kaggle/working/data/

Example data directory is: /kaggle/working/data


100%|████████████████████████████████████████| 725k/725k [00:00<00:00, 962MB/s]
Extracting 'masks/mask_0000.tif' from '/kaggle/working/data/hela-ctc-01-gt.zip' to '/kaggle/working/data/hela-ctc-01-gt.zip.unzip'
Extracting 'masks/mask_0001.tif' from '/kaggle/working/data/hela-ctc-01-gt.zip' to '/kaggle/working/data/hela-ctc-01-gt.zip.unzip'
Extracting 'masks/mask_0002.tif' from '/kaggle/working/data/hela-ctc-01-gt.zip' to '/kaggle/working/data/hela-ctc-01-gt.zip.unzip'
Extracting 'masks/mask_0003.tif' from '/kaggle/working/data/hela-ctc-01-gt.zip' to '/kaggle/working/data/hela-ctc-01-gt.zip.unzip'
Extracting 'masks/mask_0004.tif' from '/kaggle/working/data/hela-ctc-01-gt.zip' to '/kaggle/working/data/hela-ctc-01-gt.zip.unzip'
Extracting 'masks/mask_0005.tif' from '/kaggle/working/data/hela-ctc-01-gt.zip' to '/kaggle/working/data/hela-ctc-01-gt.zip.unzip'
Extracting 'masks/mask_0006.tif' from '/kaggle/working/data/hela-ctc-01-gt.zip' to '/kaggle/working/data/hela-ctc-01-gt.zip.unzip'
Ext

## Let's create our necessary functionality

With reference to our suggestion for batched evaluation of the Segment Anything models (see our [documentation](https://github.com/computational-cell-analytics/micro-sam/tree/master/finetuning/evaluation#how-to-run-the-evaluation-scripts-on-your-own-data) for details) on our own data, the first step is to create a convenience function to wrap the input images and respective instance labels around a function.

In [9]:
def get_paths(split=None):
    """For this dataset, we have a total of 84 images.
    
    We choose:
        - The first 5 images as the validation images, for grid search (relevant for AMG ans AIS)
        - The next 79 images as the test images, for running the inference(s).
    """
    image_paths = sorted(glob(os.path.join(image_dir, "*")))
    gt_paths = sorted(glob(os.path.join(segmentation_dir, "*")))
    
    if split is None:
        return image_paths, gt_paths

    else:
        if split == "val":
            return image_paths[:1], gt_paths[:1]
        elif split == "test":
            return image_paths[5:], gt_paths[5:]
        else:
            raise ValueError(f"'{split}' is not a valid split name.")

## Let's visualize how our samples look

In [10]:
# Convenience function to plot images side-by-side
def plot_samples(image, gt, segmentation=None):
    n_images = 2 if segmentation is None else 3
    fig, ax = plt.subplots(1, n_images, figsize=(10, 10))
    
    ax[0].imshow(_enhance_image(image, do_norm=False), cmap="gray")
    ax[0].axis("off")
    ax[0].set_title("Image")
    
    ax[1].imshow(gt, cmap=get_random_colors(gt), interpolation="nearest")
    ax[1].axis("off")
    ax[1].set_title("Ground Truth")
    
    if n_images == 3:
        ax[2].imshow(segmentation, cmap=get_random_colors(segmentation), interpolation="nearest")
        ax[2].axis("off")
        ax[2].set_title("Prediction")

In [ ]:
image_paths, segmentation_paths = get_paths(split="test")

for image_path, segmentation_path in zip(image_paths, segmentation_paths):
    image = imageio.imread(image_path)
    segmentation = imageio.imread(segmentation_path)

    plot_samples(image=image, gt=segmentation)
    
    break  # comment this out in case you want to visualize all the images

AMG

In [12]:
val_image_paths, val_gt_paths = get_paths(split="val")
test_image_paths, test_gt_paths = get_paths(split="test")

In [13]:
checkpoint=None  # overwrite in case you have custom checkpoints to load models from
model_type = "vit_b"  # overwrite with your desired choice of model
experiment_folder = os.path.join(root_dir, "experiment_dir", "DIC-C2DH-HeLa", model_type)

In [14]:
prediction_folder = inference.run_amg(
    checkpoint=checkpoint,
    model_type=model_type,
    experiment_folder=experiment_folder,
    val_image_paths=val_image_paths,
    val_gt_paths=val_gt_paths,
    test_image_paths=test_image_paths,
    iou_thresh_values=[0.5],
    stability_score_values=[0.5],
)

Run instance segmentation grid-search: 100%|██████████| 1/1 [00:18<00:00, 18.69s/it]


Best grid-search result: 0.0359734619541288 with parmeters:
 pred_iou_thresh = 0.5, stability_score_thresh = 0.5



Run inference for automatic mask generation: 100%|██████████| 79/79 [19:57<00:00, 15.16s/it]


Now, let's evaluate our AMG predictions from the default Segment Anything model.

In [15]:
print("Evaluating", prediction_folder)
prediction_paths = sorted(glob(os.path.join(prediction_folder, "*")))
save_path = os.path.join(experiment_folder, "results", "amg.csv")
res = run_evaluation(
    gt_paths=test_gt_paths,
    prediction_paths=prediction_paths,
    save_path=save_path,
)
print(res)

Evaluating /kaggle/working/experiment_dir/DIC-C2DH-HeLa/vit_b/amg/inference


Evaluate predictions: 100%|██████████| 79/79 [00:02<00:00, 33.25it/s]

        msa      sa50      sa75
0  0.059087  0.096867  0.061251


Visualize the AMG results

In [16]:
# TODO: visualize the AMG predictions from default SAM models.

Next, AMG from finetuned models

In [17]:
checkpoint=None  # overwrite in case you have custom checkpoints to load models from
model_type = "vit_b_lm"  # overwrite with your desired choice of model
experiment_folder = os.path.join(root_dir, "experiment_dir", "DIC-C2DH-HeLa", model_type)

In [18]:
prediction_folder = inference.run_amg(
    checkpoint=checkpoint,
    model_type=model_type,
    experiment_folder=experiment_folder,
    val_image_paths=val_image_paths,
    val_gt_paths=val_gt_paths,
    test_image_paths=test_image_paths,
    iou_thresh_values=[0.5],
    stability_score_values=[0.5],
)

Run instance segmentation grid-search: 100%|██████████| 1/1 [00:05<00:00,  5.90s/it]


Best grid-search result: 0.1927906946144028 with parmeters:
 pred_iou_thresh = 0.5, stability_score_thresh = 0.5



Run inference for automatic mask generation: 100%|██████████| 79/79 [07:50<00:00,  5.95s/it]


Quantitative for AMG micro-sam models

In [19]:
print("Evaluating", prediction_folder)
prediction_paths = sorted(glob(os.path.join(prediction_folder, "*")))
save_path = os.path.join(experiment_folder, "results", "amg.csv")
res = run_evaluation(
    gt_paths=test_gt_paths,
    prediction_paths=prediction_paths,
    save_path=save_path,
    verbose=True,
)
print(res)

Evaluating /kaggle/working/experiment_dir/DIC-C2DH-HeLa/vit_b_lm/amg/inference


Evaluate predictions: 100%|██████████| 79/79 [00:02<00:00, 31.87it/s]

        msa      sa50      sa75
0  0.195947  0.270297  0.217326


Qualitative for AMG micro-sam models

In [20]:
# TODO

AIS for finetuned models

In [21]:
prediction_folder = inference.run_instance_segmentation_with_decoder(
    checkpoint=checkpoint,
    model_type=model_type,
    experiment_folder=experiment_folder,
    val_image_paths=val_image_paths,
    val_gt_paths=val_gt_paths,
    test_image_paths=test_image_paths,
)

Run instance segmentation grid-search: 100%|██████████| 1/1 [01:00<00:00, 60.44s/it]


Best grid-search result: 0.5631257631257631 with parmeters:
 center_distance_threshold = 0.3, boundary_distance_threshold = 0.3, distance_smoothing = 1.0, min_size = 50.0



Run inference for automatic mask generation: 100%|██████████| 79/79 [00:18<00:00,  4.18it/s]


Quantitative AIS

In [22]:
print("Evaluating", prediction_folder)
prediction_paths = sorted(glob(os.path.join(prediction_folder, "*")))
save_path = os.path.join(experiment_folder, "results", "instance_segmentation_with_decoder.csv")
res = run_evaluation(
    gt_paths=test_gt_paths,
    prediction_paths=prediction_paths,
    save_path=save_path
)
print(res)

Evaluating /kaggle/working/experiment_dir/DIC-C2DH-HeLa/vit_b_lm/instance_segmentation_with_decoder/inference


Evaluate predictions: 100%|██████████| 79/79 [00:02<00:00, 32.67it/s]

        msa      sa50      sa75
0  0.624173  0.774011  0.738191


Qualitative AIS

In [23]:
# TODO

Interactive Segmentation

First, using default SAM models

In [24]:
checkpoint=None  # overwrite in case you have custom checkpoints to load models from
model_type = "vit_b"  # overwrite with your desired choice of model
experiment_folder = os.path.join(root_dir, "experiment_dir", "DIC-C2DH-HeLa", model_type)

predictor = get_sam_model(model_type=model_type, checkpoint_path=checkpoint)

In [25]:
prediction_dir = os.path.join(experiment_folder, "start_with_point_prompt")
embedding_folder = os.path.join(experiment_folder, "embeddings")

inference.run_inference_with_iterative_prompting(
    predictor=predictor,
    image_paths=test_image_paths,
    gt_paths=test_gt_paths,
    embedding_dir=embedding_folder,
    prediction_dir=prediction_dir,
    start_with_box_prompt=False,  # overwrite to start with the box prompt
    use_masks=False,  # overwrite to use logits masks for iterative prompting
)

Run inference with iterative prompting for all images: 100%|██████████| 79/79 [01:37<00:00,  1.23s/it]


Quantitative for points

In [26]:
run_evaluation_for_iterative_prompting(
    gt_paths=test_gt_paths,
    prediction_root=prediction_dir,
    experiment_folder=experiment_folder,
    start_with_box_prompt=False,
)

Evaluating iteration00


Evaluate predictions: 100%|██████████| 79/79 [00:02<00:00, 33.59it/s]


        msa      sa50      sa75
0  0.185422  0.259831  0.214434
Evaluating iteration01


Evaluate predictions: 100%|██████████| 79/79 [00:02<00:00, 32.84it/s]


        msa      sa50      sa75
0  0.316831  0.519859  0.332765
Evaluating iteration02


...

        msa      sa50      sa75
0  0.441917  0.714746  0.473958
Evaluating iteration06


Evaluate predictions: 100%|██████████| 79/79 [00:02<00:00, 32.16it/s]


        msa     sa50      sa75
0  0.455805  0.72714  0.496399
Evaluating iteration07


Evaluate predictions: 100%|██████████| 79/79 [00:02<00:00, 32.97it/s]

        msa     sa50      sa75
0  0.456492  0.73958  0.497296
